# Introduction

This notebook guides through calibration function development using published data (S. De Vito et al., Sensors and Actuators B: Chemical, Volume 129, Issue 2).

In [1]:
# Load libriaries.
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, explained_variance_score, mean_squared_error
import time

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['lines.linewidth'] = 1.0
plt.rcParams['font.size'] = 6.0
plt.rcParams['axes.titlesize'] = 6.0

In [2]:
# Define helper functions.
scaler = StandardScaler(copy=True) # Define standard scaler instance.
imputer = SimpleImputer() # Define imputer instance.
detector = IsolationForest(n_estimators=1000, random_state=0) # Define outlier detector instance.

# Data Handling
Load data to pandas data frame.

In [3]:
# Load data and keep only first six months.
data = pd.read_excel("data.xlsx")
data = data[data["Date"] <= "2004-09-10"]

# Visualize data summary.
data.describe()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
count,3447.000000,4275.000000,914.000000,4275.000000,4275.000000,3525.000000,4275.000000,3522.000000,4275.000000,4275.000000,4275.000000,4275.000000,4275.000000
mean,1.918422,1077.334113,218.811816,9.850661,937.848090,126.051348,897.271150,94.706133,1617.936394,944.460370,23.451201,42.485045,1.153040
std,1.206673,200.344215,204.459921,6.566329,240.489984,83.970746,228.686816,36.637686,255.788785,343.175279,7.843419,15.985736,0.343338
min,0.100000,708.000000,7.000000,0.464418,437.000000,2.000000,387.500000,5.000000,955.000000,263.000000,6.100000,9.175000,0.375444
25%,1.000000,932.250000,67.000000,4.843458,755.500000,63.000000,735.875000,67.000000,1441.750000,692.875000,17.300000,29.500000,0.904255
50%,1.700000,1047.500000,150.000000,8.499723,919.500000,108.000000,860.000000,94.000000,1587.000000,896.500000,23.450000,42.500000,1.096318
75%,2.500000,1184.250000,297.000000,13.385657,1096.500000,168.000000,1022.750000,119.000000,1752.625000,1150.625000,28.600000,54.000000,1.366492
max,8.100000,2039.750000,1189.000000,40.260061,1776.250000,631.000000,1940.750000,233.000000,2746.000000,2474.750000,44.600000,85.150002,2.180639


# Preprocessing
The data is preprocessed and sliced into different sets. Data is scaled before analysis. In particular, standard scaling provides better results with respect to R2 score and explained variance.
No imputation is performed.

In [4]:
# Select columns and remove rows with missing values.
columns = ["PT08.S1(CO)", "PT08.S3(NOx)", "PT08.S4(NO2)", "PT08.S5(O3)", "T", "AH", "CO(GT)", "C6H6(GT)", "NOx(GT)", "NO2(GT)"]
data = data[columns].dropna(axis=0)

# Scale data to zero mean and unit variance.
X_t = scaler.fit_transform(data)

# Optional: Impute missing values.
# X_t = imputer.fit_transform(X_t)

# Remove outliers.
is_inlier = detector.fit_predict(X_t)
X_t = X_t[(is_inlier > 0),:]

# Restore frame.
data = pd.DataFrame(X_t, columns=columns)

In [5]:
# Reassign training data.
X = data[["PT08.S1(CO)", "PT08.S3(NOx)", "PT08.S4(NO2)", "PT08.S5(O3)", "T", "AH"]]
Y = data[["CO(GT)", "C6H6(GT)", "NOx(GT)", "NO2(GT)"]]

# Perform slicing into training and test set.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=0)
print("Training set consists of", X_train.shape[0], "samples. Test set contains", X_test.shape[0], "samples.")

Training set consists of 1892 samples. Test set contains 811 samples.


# Results and Discussion

## Neural Network

In [6]:
from Regression.NeuralNetworkRegressor import NeuralNetworkRegressor
nnr = NeuralNetworkRegressor(alpha=1.0, batch_size=300, learning_rate=1e-2, n_iterations=1e4)
start = time.time()
nnr.fit(X_train.values, Y_train.values)
stop = time.time()
print("Computational time was:", np.round(stop-start, 2), "s.")

Computational time was: 2.01 s.


In [7]:
# Evaluate model in testing phase.
Y_predict = nnr.predict(X_test) # Predict.
exp = explained_variance_score(Y_test, Y_predict) 
r2 = r2_score(Y_test, Y_predict)
mse = mean_squared_error(Y_test, Y_predict)
print("Explained variance is", np.round(exp, 2), ", R2 score is", np.round(r2, 2), ", and MSE is", np.round(mse, 2), ".")

Explained variance is 0.83 , R2 score is 0.83 , and MSE is 0.1 .


## Linear Regression

In [8]:
from Regression.LinearRegressor import LinearRegressor
lr = NeuralNetworkRegressor(alpha=1.0, batch_size=300, learning_rate=1e-2, n_iterations=1e4)
start = time.time()
lr.fit(X_train.values, Y_train.values)
stop = time.time()
print("Computational time was:", np.round(stop-start, 2), "s.")

Computational time was: 2.38 s.


In [9]:
# Evaluate model in testing phase.
Y_predict = lr.predict(X_test) # Predict.
exp = explained_variance_score(Y_test, Y_predict) 
r2 = r2_score(Y_test, Y_predict)
mse = mean_squared_error(Y_test, Y_predict)
print("Explained variance is", np.round(exp, 2), ", R2 score is", np.round(r2, 2), ", and MSE is", np.round(mse, 2), ".")

Explained variance is 0.83 , R2 score is 0.83 , and MSE is 0.1 .
